# Hugging Face Library 'Transformer'およびT5Tokenizerのダウンロード

参考(https://qiita.com/takubb/items/fd972f0ac3dba909c293)これを基に改造し、最新のGoogle Colaboratoryで動作するようにした

In [1]:
!export CUDA_LAUNCH_BLOCKING=1
# !pip install torch
!pip install torchvision
!pip install transformers
#!apt install swig
# Sentencepieceのインストール
!pip install sentencepiece
!pip install mecab-python3
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
import gzip
import shutil
import sqlite3
import random
from math import ceil
import re
import csv
import glob
import torchvision
import statistics
import numpy as np
import lightgbm
import pickle
import statistics
import MeCab
import re
import copy

import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# PyTorchとGPU設定

In [3]:
#!pip install torch
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Data Augmentation kansuu

In [4]:
# synreplace - replace kasho kosuu
# randinsert - tasu kotoba no kazu
# randdelete - delete kakuritsu
# randswap - swap kaisuu

class synreplace(object):
    def __init__(self, num):
        self.num = num
        self.model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
    def __call__(self, textlist):
        # textlist: honbun no list
        textlen = torch.where(textlist == 3)[0][0]
        for n in range(self.num):
            # chikan shiro
            masked_idx = random.randint(2, textlen-1)
            textlist[masked_idx] = 6
            # convert to tensor
            token_tensor = torch.tensor(textlist)
            # get the top 10 predictions of the masked token
            self.model = self.model.eval()
            with torch.no_grad():
                outputs = self.model(torch.unsqueeze(token_tensor, 0))
                predictions = outputs[0][0, masked_idx].topk(1)
            for i, index_t in enumerate(predictions.indices):
                index = index_t.item()
            textlist[masked_idx] = index
        return textlist

class randinsert(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        for n in range(self.num):
            insword = textlist[random.randint(1,len(textlist)-1)]
            i = random.randint(1,len(textlist)-1)
#            print('len: ', len(textlist))
#            print(i)
            while textlist[i] == 3:
                i = random.randint(1,len(textlist)-1)
#                print(i)
            textlist = torch.cat([textlist[0:i], torch.tensor([insword]), textlist[i:-1]])
        return textlist

class randdelete(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
#        print(textlist.shape)
        for i in range(3,len(textlist)-1):
            if textlist[i] == 3:
                continue
            r = random.uniform(0, 1)
            if r < self.num:
#                textlist.pop(i)
                textlist = torch.cat([textlist[0:i], textlist[i+1:], torch.tensor([3])])
#                print(textlist)
        return textlist

class randswap(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        counter = 0
        #rs_sents = np.zeros(len(textlist), dtype=object)
        for i in range(len(textlist)):
            while self.num > counter:
                box = 0
                random_idx_1 = random.randint(1, len(textlist)-1)
                while textlist[random_idx_1] == 3:
                    random_idx_1 = random.randint(0, len(textlist)-1)
                random_idx_2 = random.randint(1, len(textlist)-1)
                while random_idx_1 == random_idx_2 or textlist[random_idx_2] == 3:
                    random_idx_2 = random.randint(0, len(textlist)-1)
                    # print(random_idx_1, random_idx_2)
                box = textlist[random_idx_1]
                textlist[random_idx_1] = textlist[random_idx_2]
                textlist[random_idx_2] = box
                counter += 1
        return textlist

# Custom Tensor Dataset
https://stackoverflow.com/questions/55588201/pytorch-transforms-on-tensordataset

# データセットの準備

ライブドアニュースコーパスをダウンロード

    ダウンロードしたファイルは圧縮（tar.gz形式）ファイル
    様々なジャンル（IT,スポーツ,家電,映画など）のWEBメディアごとにフォルダに記事がテキストファイルで保存されている
    
以下、ファイルを読み込んで、必要な部分を抽出

In [5]:
#urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")
# ダウンロードした圧縮ファイルのパスを設定
#tgz_fname = "ldcc-20140209.tar.gz" 
# 2つをニュースメディアのジャンルを選定
mydata = '/export/livedoor' 
#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 

def remove_brackets(inp):
    output = re.sub(u'[〃-〿]', '',(re.sub('＝|=|×|\(|\)|“|”|（|）|／|\[|\]| |　|…|・|\n|\t|/|＜|＞|@|＠', '', re.sub(u'[ℊ-⿻]', '', inp)))) #210A ~ 2FFF
    return output

"""
def read_url(f):
    url = next(f)
    return url[:-1]

def read_date(f):
    date = next(f)
    date = remove_brackets(date.encode().decode('utf-8'))
    return date[:-1]
"""

def read_title(f):
    next(f)
    next(f)
    title = next(f)
    title = remove_brackets(title.encode().decode('utf-8'))
    return title[:-1]

def read_para(f):
    p = ''
    while True:
        try:
            para = next(f)
            para = remove_brackets(para.encode().decode('utf-8'))
            p += para
        except StopIteration:
            break
    return p [:-1]

In [6]:
#directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/it-life-hack']
#target_genre = ["dokujo-tsushin", "it-life-hack"] 
directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/peachy']
target_genre = ["dokujo-tsushin", "peachy"] 

In [7]:
zero_fnames = []
one_fnames = []

if os.path.exists(tsv_fname) == True:
    with open(tsv_fname, "r+") as f:
        f.truncate(0)

for i in range(2):
    for filename in os.listdir(directory[i]):
        if "LICENSE.txt" in filename:
            continue
        f = os.path.join(directory[i], filename)
        #if os.path.isfile(f):
        #    print(f)
        if target_genre[0] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[0], '0', title, para]
                    writer.writerow(row)
            continue
        if target_genre[1] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[1], '1', title, para]
                    writer.writerow(row)
            continue

pandasでデータを読み込み

In [8]:
import pandas as pd
# データの読み込み
df = pd.read_csv("all_text.tsv", 
                 delimiter='\t', header=None, names=['media_name', 'label','title','sentence'])
# データの確認
print(f'データサイズ： {df.shape}')
df.sample(10)

データサイズ： (1712, 4)


,media_name,label,title,sentence
38,dokujo-tsushin,0,家に帰ったらダメ人間化!?人には見せられない私の,綾瀬はるか主演のドラマホタルノヒカリ2。主人公の雨宮蛍は、バリバリのキャリアウーマンだが、夜...
1375,peachy,1,リゾートホテルで極上の快眠体験！お疲れガールにオススメの宿泊プラ,あなたはどんな美容液やサプリメントよりも効果のある、最高の美容法を知っていますか？その答えは...
1362,peachy,1,韓国女子の美容ケアはデリケート部分から、専用クレンジングも豊,韓国では女性のデリケート部分専用の洗浄剤女性清潔剤が多数登場しています。夏になると気温が高く...
954,peachy,1,300点が展示されるバービー展で、ルブタンバービーを限定販売招待券プレゼン,女子なら誰でも子供時代に夢中になったファッションドールのバービー人形。皆さんも抜群のプロポー...
392,dokujo-tsushin,0,シングル女性が陥りやすい友達親子リスクと,自分でもまったく違和感なく使っていたのだが、友人との会話の中で自分の母親を登場させる時母がで...
335,dokujo-tsushin,0,淀殿はタバコ顔だった！？戦国時代に学ぶ健康,毎週大河ドラマをみている歴女の佐知恵さん37歳は、茶を演じる女優が美人なのはおかしいという。...
840,dokujo-tsushin,0,変わらないねのひと言に感じるコ,お正月で実家に帰省したとき、高校時代の同級生に久しぶりに会ったんですけど、学生時代からゼンゼ...
1148,peachy,1,インタビュー：植松晃士さん黒目の印象を変えて、脱変わり映えのしない,美しくなりたい女性にとって、日の研究テーマは瞳をどう印象的に見せるか。アイラインをしっかりと...
1322,peachy,1,オフィス使いもできるエコ＆フェアトレードなデザイナーズアクセサリ,発展途上国の生産者とフェアな条件での取引を通じて、生産者や労働者の生活改善と自立を目指す運動...
794,dokujo-tsushin,0,女性が抱えるアノ悩みは、身近な食物と食べ方に解決の光明あ,美容や健康に関する興味は人一倍強い独女達にとっては、目を向けざるを得ない興味深いニュースが伝...


//文章データをsentences、ラベルデータを labelsに保存、以降この2変数だけを利用

In [9]:
mn = df.media_name.values
labels = df.label.values
titles = df.title.values
sentences = df.sentence.values

In [10]:
tagger = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
  # MeCabで分かち書きを行う
    sentence = tagger.parse(sentence)
  # 半角全角英数字などは削除する
#    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
  # 記号なども削除する
#    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
  # スペース区切で形態素の配列に変換する
    wakati = sentence.split(" ")
  # 空要素を削除する
    wakati = list(filter(("").__ne__, wakati))
    return wakati

In [11]:
wakati_sentences = []

for i in range(len(sentences)):
    wakati_sentences.append(make_wakati(sentences[i]))

In [12]:
wcount = 256

emptylist = []
ssentences = np.append(emptylist, copy.deepcopy(sentences))

emplist = []
sectionlist = []

for i in enumerate(wakati_sentences):
    emp = 0
    section = 1
    if len(i[1])>wcount:
        wcount = 128
        count = 0
        countend = 0
        ssentences[i[0]] = []
        while len(i[1])-count-wcount>0:
            oneph = ''
            countend_ = 1
            while countend_%wcount != 0:
                oneph += i[1][countend]
                countend+=1
                countend_+=1
            ssentences[i[0]].append(oneph)
            count += wcount-1
            section += 1
        oneph = ''
        for j in range(len(i[1][count:-1])):
            oneph += i[1][count]
            count += 1
            emp += 1
        emplist.append(emp)
        ssentences[i[0]].append('')
        ssentences[i[0]][-1] = oneph
        sectionlist.append(section)
    else:
        oneph = ''
        for k in range(len(i[1])):
            oneph += i[1][k]
            ssentences[i[0]] = oneph
        emp = wcount - len(i[1])
        emplist.append(emp)
        sectionlist.append(1)

In [13]:
ssentences[0]

['あなたはどんな人物？何を目指している？あなたにとって一番大切なものは何？庶民でありながらも成績優秀で大学を卒業し、エリートコースまっしぐらの生活を送るため、ゴールドマンサックスの面接にやってきたアニースカーレットヨハンソン。簡単な質問だわと答えるも、言葉が出てこない。母親の期待に応えるため、ひたすら勝ち組になるために頑張って来たアニーにとって、エリートコースはあくまでも母が決めた道。自分のことが、分からない！！なんとなく大学に行き、なんとなく就職し、なん',
 'となく年を取ってしまった。自分を見失う瞬間は、誰にでも必ずあるもの。アニーのように初めは20代でやってくるのかもしれない。でもきっと、30代、40代、50代になっても、同じように立ち止まる瞬間がやってくる。自分の人生、このままでいいのだろうか？いまの自分が本当の自分なのだろうか？自分にとっての幸せとは何なのだろうか？自分探しと言えば、旅行や留学が定番ですが、彼女が選んだのは、',
 'いまの自分とかけ離れた世界職業に身を置いてみるという方法。優秀な成績で大学を卒業した人がなかなか就く事はないナニーベビーシッターよりは乳母に近いという職業。でもただのナニーではなく、上流階級の家のナニーというところが面白い！自分が目指していたエリート階級の生活を近くで観察できる状況に置かれたことで、自分が大切にしなければならないものに気付いていく。一番心に残っているのは、アニーが雇い主のミセスXのことを友達に話すシーン',
 '。ミセスXは何もかも手に入れているのに、ちっとも幸せそうじゃないのというアニーに対して、あなたと彼女は似た者同士よという友達の言葉の深さに、ハッとさせられます。お金持ちと結婚し、子供を授かり、豪邸に住み、ショッピングにエステ三昧のミセスX。端から見たら、誰もが羨む憧れのセレブリティ。でも彼女が幸せじゃないのは、何を持っていれば幸せなのかが分かっていないから。結局はいくらお金があっても、仕事',
 'が成功しても、結婚出産というステータスがあっても、自分の人生にとって何が一番大切なものなのかが分かっていなければ、幸せにはなれないという、とても厳しいメッセージ。いま自分が目指しているものは、肩書きだけ、表面的なものではないか？アニーの立場からも、ミセスXの立場からも、考えさせられます。自分が進んできた道

# BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
単語分割とIDへ変換

# テスト実行

In [14]:
w_input_ids = []
w_attention_masks = []

for sent in ssentences:
    p_input_ids = []
    p_attention_masks = []
    for sect in sent:
        sencoded_dict = tokenizer.encode_plus(
                            sect,                      
                            add_special_tokens = True, # Special Tokenの追加
                            max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                            truncation=True,                
                            pad_to_max_length = True,# PADDINGで埋める
                            return_attention_mask = True,   # Attention maksの作成
                            return_tensors = 'pt',     #  Pytorch tensorsで返す
                       )
        p_input_ids.append(torch.tensor(sencoded_dict['input_ids']).view(-1))
        p_attention_masks.append(torch.tensor(sencoded_dict['attention_mask']).view(-1))
    w_input_ids.append(p_input_ids)
    w_attention_masks.append(p_attention_masks)

In [15]:
# nagasa soroeru yo - id
pad = torch.full((1,130),3).view(-1)
maxlen = max(sectionlist)

for i in range(len(w_input_ids)):
    if maxlen>len(w_input_ids[i]):
        while maxlen>len(w_input_ids[i]):
            w_input_ids[i].append(pad)

In [16]:
# nagasa soroeru yo - attention
pad = torch.full((1,130),0).view(-1)

for i in range(len(w_attention_masks)):
    if maxlen>len(w_attention_masks[i]):
        while maxlen>len(w_attention_masks[i]):
            w_attention_masks[i].append(pad)

In [17]:
len(w_input_ids)
#len(sectionlist)

1712

In [18]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split, RandomSampler, SequentialSampler
import torchvision.transforms as transforms
from transformers import RobertaForMaskedLM
import random

# データローダーの作成
data_transform = transforms.Compose([
    synreplace(1),
    randinsert(3),
    randdelete(0.15), 
    randswap(2),
    transforms.ToTensor()
])

class MyDatasets(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels, valids, transform=None):
        self.ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.valids = valids
        self.transform = transform
        
    def __getitem__(self, idx):
        xa, mask, label, valid = self.ids[idx], self.attention_mask[idx], self.labels[idx], self.valids[idx]
        if self.transform:
            xa = self.transform(xa)
        return xa, mask, [label]*len(xa), valid

    def __len__(self):
        return len(self.ids)

    # 80%地点のIDを取得
num_dataset = len(w_input_ids)
#train_size = 50
#train_size = 100
#train_size = 200
#train_size = 500
train_size = 1000
val_size = num_dataset - train_size

print('訓練データ数:{}'.format(train_size))
print('検証データ数:{}'.format(val_size))

# データセットクラスの作成 ichigyoume ha randamu shitei
indices = np.random.choice(num_dataset, num_dataset, replace=False)
wt_input_ids = [w_input_ids[i] for i in indices[:train_size]]
wt_attention_masks = [w_attention_masks[i] for i in indices[:train_size]]
wt_labels = [labels[i] for i in indices[:train_size]]
#wt_values = [value[i] for i in indices[:train_size]]
wt_values = [sectionlist[i] for i in indices[:train_size]]
wv_input_ids = [w_input_ids[i] for i in indices[train_size:]]
wv_attention_masks = [w_attention_masks[i] for i in indices[train_size:]]
wv_labels = [labels[i] for i in indices[train_size:]]
#wv_values = [value[i] for i in indices[train_size:]]
wv_values = [sectionlist[i] for i in indices[train_size:]]

train_dataset = MyDatasets(wt_input_ids, wt_attention_masks, wt_labels, wt_values)
val_dataset = MyDatasets(wv_input_ids, wv_attention_masks, wv_labels, wv_values)

# データローダーの作成
batch_size = 50

'''
# 訓練データローダー
# shuffle True/False to compare or not
train_dataloader = DataLoader(
            train_dataset,
            batch_size = batch_size,
            shuffle = True
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            batch_size = val_size,
            shuffle = False
        )
'''

訓練データ数:1000
検証データ数:712


'\n# 訓練データローダー\n# shuffle True/False to compare or not\ntrain_dataloader = DataLoader(\n            train_dataset,\n            batch_size = batch_size,\n            shuffle = True\n        )\n\n# 検証データローダー\nvalidation_dataloader = DataLoader(\n            val_dataset, \n            batch_size = val_size,\n            shuffle = False\n        )\n'

# LGBM Dataset

In [19]:
lwt_ids = []
lwv_ids = []

for i in range(len(wt_input_ids)):
    article = []
    for j in range(len(wt_input_ids[i])):
        article.append(wt_input_ids[i][j].detach().numpy())
#        thisblock = wt_input_ids[i][j].detach().numpy()
#        for k in thisblock:
#            article.append(k)
    lwt_ids.append(article)
    
''' UGOKAN KEDO WAKARAN
for i in range(len(wt_input_ids)):
    article = []
    for j in range(len(wt_input_ids[i])):
        if j == 0:
            pass
        if j == 1:
            block = torch.cat((wt_input_ids[i][j-1], wt_input_ids[i][j]), dim=1).view(-1)
        else:
            block = torch.cat((block, wt_input_ids[i][j]), dim=1).view(-1)
    lwt_ids.append(article)
'''

for i in range(len(wv_input_ids)):
    article = []
    for j in range(len(wv_input_ids[i])):
        article.append(wv_input_ids[i][j])
    lwv_ids.append(article)

class MyLGBMDatasets(torch.utils.data.Dataset):
    def __init__(self, input_ids, labels, transform=None):
        self.ids = input_ids
        self.labels = labels
        self.transform = transform
        
    def __getitem__(self, idx):
        xa, label = self.ids[idx], self.labels[idx]
        # Transform suru imi aru?
#        if self.transform:
#            xa = self.transform(xa)
        return xa, label

    def __len__(self):
        return len(self.ids)

l_train_dataset = MyLGBMDatasets(lwt_ids, wt_labels)
l_val_dataset = MyLGBMDatasets(lwv_ids, wv_labels)

In [20]:
l_train_dataset = np.array(l_train_dataset)
l_val_dataset = np.array(l_val_dataset)
l_train_dataset[1][1]

1

In [21]:
ltd_list = []
lvd_list = []

for i in range(len(l_train_dataset)):
    block = []
    for j in range(len(l_train_dataset[i][0])):
        for k in range(len(l_train_dataset[i][0][j])):
            block.append(l_train_dataset[i][0][j][k].item())
    ltd_list.append(block)
    
for i in range(len(l_val_dataset)):
    block = []
    for j in range(len(l_val_dataset[i][0])):
        for k in range(len(l_val_dataset[i][0][j])):
            block.append(l_val_dataset[i][0][j][k].item())
    lvd_list.append(block)

In [22]:
ltd_label = []
lvd_label = []

for i in range(len(l_train_dataset)):
    ltd_label.append(l_train_dataset[i][1])
    
for i in range(len(l_val_dataset)):
    lvd_label.append(l_val_dataset[i][1])


# LGBM

In [23]:
# Xtrain = l_train_dataset[:][0]  # kore to 
# Xtest = l_val_dataset[:][0]     # kore

Xtrain = ltd_list  # kore to 
Xtest = lvd_list     # kore
ytrain = ltd_label
ytest = lvd_label

X_train = np.array(Xtrain).copy()
X_test = np.array(Xtest).copy()
y_train = np.array(ytrain).copy()
y_test = np.array(ytest).copy()

In [30]:
y_test.ndim

1

In [24]:
lgb_clf = lightgbm.LGBMClassifier(max_depth=10, num_iterations=30)
eval_res = {}
lgb_clf.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=5)

ValueError: setting an array element with a sequence.

In [ ]:
l_preds = lgb_clf.predict(X_test)

In [ ]:
print('accuracy: ', accuracy_score(l_preds, y_test))
print('f1 score: ', f1_score(l_preds, y_test))

In [ ]:
lightgbm.plot_metric(lgb_clf)

In [ ]:
'''
def accuracy(pdf):
    cor = 0
    for i in range(len(wv_labels)):
        if pdf[i] == label_df.values[i]:
            cor += 1
    acc = 100*cor/len(pdf)
    return acc

def fscore(pdf):
    cor = 0
    ypnum = 0 #yosoku
    spnum = 0 #seikai
    pnum = 0
    rnum = 0
    fone = 0
    for i in range(len(wv_labels)):
        if pdf[i] == label_df.values[i]:
            cor += 1
        if pdf[i] == 0:
            ypnum += 1
            if label_df.values[i] == 0:
                pnum += 1
        if label_df.values[i] == 0:
            spnum += 1
            if pdf[i] == 0:
                rnum += 1
    # tekigou
    tp = pnum/ypnum
    # saigen
    sp = rnum/spnum
    # f1
    fone = (tp*sp)/(tp+sp)
    return fone
'''